# IMPORTS

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import dayofweek, col

from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder

import numpy as np
import matplotlib.pyplot as plt

# LOAD DATA

You can download the data at [Kaggle site](https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023?select=2021.csv) or through the provided S3 bucket location.

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("FlightDelayPrediction").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
file_path = "2021.csv"
data_2021 = spark.read.csv(file_path, header=True, inferSchema=True)

In [ ]:
target_col = "DEP_DELAY"

# DATA ANALYSIS

In [ ]:
data_2021.printSchema()

In [ ]:
data_2021.show(5, truncate=False)

In [ ]:
data_2021.describe().show()

In [ ]:
for df_col in data_2021.columns:
    missing_count = data_2021.filter(data_2021[df_col].isNull()).count()
    print(f"{df_col}: {missing_count} missing values")

# DATA PREPROCESSING / FEATURE SELECTION

In [ ]:
# Sort data by FL_DATE column in ascending order
preprocessed_data_2021 = data_2021.sort("FL_DATE")

# Remove rows that have cancelled flights
preprocessed_data_2021 = preprocessed_data_2021.filter(data_2021.CANCELLED == 0)

# Add column for day of the week, based on FL_DATE
preprocessed_data_2021 = preprocessed_data_2021.withColumn("DAY_OF_WEEK", dayofweek(col("FL_DATE")))

In [ ]:
# Keep these columns in a new dataframe (Domain Knowledge Feature Selection)
preprocessed_data_2021 = preprocessed_data_2021.select("FL_YEAR", "FL_MONTH", "FL_DAY", "DAY_OF_WEEK", "AIRLINE_CODE", "ORIGIN", "DEST", "CRS_DEP_TIME", target_col)

preprocessed_data_2021 = preprocessed_data_2021.dropna()

In [ ]:
preprocessed_data_2021.count()

In [ ]:
# Convert categorical columns to OneHotEncoded vector
indexer = StringIndexer(inputCols=['AIRLINE_CODE', 'ORIGIN', 'DEST'], outputCols=['INDEXED_AIRLINE_CODE', 'INDEXED_ORIGIN', 'INDEXED_DEST'])
preprocessed_data_2021 = indexer.fit(preprocessed_data_2021).transform(preprocessed_data_2021)

encoder = OneHotEncoder(inputCols=['INDEXED_AIRLINE_CODE', 'INDEXED_ORIGIN', 'INDEXED_DEST'], outputCols=['ONEHOT_AIRLINE_CODE', 'ONEHOT_ORIGIN', 'ONEHOT_DEST'])
preprocessed_data_2021 = encoder.fit(preprocessed_data_2021).transform(preprocessed_data_2021)

In [ ]:
# Keep these onehot encoded columns only
preprocessed_data_2021 = preprocessed_data_2021.select("FL_YEAR", "FL_MONTH", "FL_DAY", "DAY_OF_WEEK", 'ONEHOT_AIRLINE_CODE', 'ONEHOT_ORIGIN', 'ONEHOT_DEST', target_col)

In [ ]:
# Take only first 10,000 rows, due to limited CPU power. If you have a better CPU or running this on AWS, you can comment this line out.
preprocessed_data_2021 = preprocessed_data_2021.limit(10000)

In [ ]:
preprocessed_data_2021.show(5, truncate=False)

In [ ]:
preprocessed_data_2021.describe().show()

In [ ]:
# Vectorization
feature_cols = list(set(preprocessed_data_2021.columns) - set([target_col])) # All columns except ARR_DELAY (target variable)
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
vectorized_data_2021 = vector_assembler.transform(preprocessed_data_2021)

In [ ]:
vectorized_data_2021.select("features", target_col).show(5, truncate=False)

In [ ]:
# Standardization
scaler = StandardScaler(withMean=True, withStd=True, inputCol="features", outputCol="scaled_features")
scaled_data_2021 = scaler.fit(vectorized_data_2021).transform(vectorized_data_2021)

In [ ]:
scaled_data_2021.select("scaled_features", target_col).show(5, truncate=False)

# TRAIN/TEST SPLIT

In [ ]:
# Define train/test split percentage
train_percent = 0.6
val_percent = 0.2
test_percent = 0.2

# Calculate the split indices
count = scaled_data_2021.count()
train_index = int(train_percent * count)
val_index = train_index + int(val_percent * count)

# Split the data into training, validation, and testing sets
train_data = scaled_data_2021.limit(train_index)
val_data = scaled_data_2021.limit(val_index).subtract(train_data)
test_data = scaled_data_2021.subtract(train_data).subtract(val_data)

# Extract train, validation, test sets from the dataset
X_train, y_train = train_data.select('scaled_features'), train_data.select(target_col)
X_val, y_val = val_data.select('scaled_features'), val_data.select(target_col)
X_test, y_test = test_data.select('scaled_features'), test_data.select(target_col)

# MODEL BUILDING

In [ ]:
class RNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = learning_rate
        self.clip_value = 0.01
        
        # Initialize weights
        self.input_hidden_weights = np.random.randn(input_size, hidden_size)
        self.hidden_output_weights = np.random.randn(hidden_size, output_size)
        starting_bias = 0.1
        self.hidden_bias = np.array([starting_bias] * hidden_size)
        self.output_bias = np.array([starting_bias] * output_size)
    
    def leaky_relu(self, x):
        return np.where(x > 0, x, 0.01 * x)
    
    def clip_gradients(self, gradients):
        return np.clip(gradients, -self.clip_value, self.clip_value)

    def forward(self, x):
        self.hidden_state = self.leaky_relu(x.dot(self.input_hidden_weights) + self.hidden_bias) 
        output = self.hidden_state.dot(self.hidden_output_weights) + self.output_bias 
        return output

    def backward(self, x, output, target):
        loss_gradient = 2 * (output - target)  # Mean Squared Error loss gradient
        hidden_state_gradient = self.hidden_output_weights.dot(loss_gradient)
        
        # Clip gradients before updating weights
        hidden_state_gradient_clipped = self.clip_gradients(hidden_state_gradient)
        loss_gradient_clipped = self.clip_gradients(loss_gradient)
    
        self.input_hidden_weights -= self.lr * np.outer(x, hidden_state_gradient_clipped)
        self.hidden_output_weights -= self.lr * np.outer(self.hidden_state, loss_gradient_clipped)
        self.hidden_bias -= self.lr * hidden_state_gradient_clipped
        self.output_bias -= self.lr * loss_gradient_clipped
        
    def predict(self, x):
        self.hidden_state = self.leaky_relu(x.dot(self.input_hidden_weights) + self.hidden_bias)
        output = self.hidden_state.dot(self.hidden_output_weights) + self.output_bias
        return output

In [ ]:
def mean_squared_error(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# TRAINING

In [ ]:
X_train_np, y_train_np = X_train.collect(), y_train.collect()

In [ ]:
X_val_np, y_val_np = X_val.collect(), y_val.collect()

In [ ]:
# Parameters
learning_rate = 0.001
epochs = 10
hidden_size = 1

input_size = len(X_train_np[0][0])
output_size = 1
model = RNN(input_size, hidden_size, output_size, learning_rate)

train_losses = []
val_losses = []

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}:')
    
    # Training evaluation
    train_loss = 0
    for i in range(len(X_train_np)):
        input_train = np.array(X_train_np[i][0])
        target_train = y_train_np[i][0]
        output_train = model.forward(input_train)
        model.backward(input_train, output_train, target_train)
        train_loss += mean_squared_error(target_train, output_train)
    avg_train_loss = train_loss / len(X_train_np)
    train_losses.append(avg_train_loss)
    print(f'Train Loss: {avg_train_loss}', end=' | ')
    
    # Validation evaluation
    val_loss = 0
    for i in range(len(X_val_np)):
        input_val = np.array(X_val_np[i][0])
        target_val = y_val_np[i][0]
        output_val = model.forward(input_val)
        val_loss += mean_squared_error(target_val, output_val)
    avg_val_loss = val_loss / len(X_val_np)
    val_losses.append(avg_val_loss)
    print(f'Validation Loss: {avg_val_loss}\n')

In [ ]:
def plot_loss(train_loss, val_loss, hidden_size):
    # Create an array for the number of epochs, e.g., [1, 2, 3, ...]
    epochs = np.arange(1, len(train_loss) + 1)

    # Plot training accuracy and validation accuracy on the same graph
    plt.plot(epochs, train_loss, label='Training Loss', marker='o', linestyle='-')
    plt.plot(epochs, val_loss, label='Validation Loss', marker='o', linestyle='-')

    # Set axis labels and a legend
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='best')

    # Display the plot
    plt.xticks(np.arange(1, len(train_loss) + 1, step=1))
    plt.title(f'h = {hidden_size} | Training and Validation Loss (by Epoch)')
    plt.show()
    
plot_loss(train_losses, val_losses, hidden_size)

# EVALUATION

In [ ]:
X_test_np, y_test_np = X_test.collect(), y_test.collect()

In [ ]:
# Make predictions on a new set of data
preds = []

for i in range(len(X_test_np)):
    input = np.array(X_test_np[i][0])
    output = model.predict(input)
    preds.append(output)

# Convert predictions to a NumPy array
preds = np.array(preds)

In [ ]:
mse_value = mean_squared_error(y_test_np, preds)
print(f"Test Mean Squared Error: {mse_value}")
print(f"Test Root Mean Squared Error: {np.sqrt(mse_value)}")